# Desarrollo del Proyecto 1 - Recomendación de películas
Hecho por: Alejandra Salas

### Data Engineering

## 1. Extraer los datos de los archivos

In [1]:
# Se importan las librerías a usar.
import numpy as np
import pandas as pd
import json
import ast

In [2]:
# Cargamos los datos que se encuentran en archivo tipo ".csv" a un dataframe.
creditos = pd.read_csv('credits.csv',sep=',')
movies =pd.read_csv('movies_dataset.csv', sep=',',low_memory=False)

In [5]:
# Visualizamos los datos del dataframe "creditos".
creditos.sample(2)

,cast,crew,id
17739,"[{'cast_id': 2, 'character': 'Himself', 'credi...","[{'credit_id': '52fe487cc3a368484e0fac8b', 'de...",72744
6035,"[{'cast_id': 1, 'character': 'Papa Reilly', 'c...","[{'credit_id': '52fe45059251416c91024cf1', 'de...",33221


In [6]:
# Visualizamos los datos del dataframe "movies".
movies.sample(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
9555,False,NaN,0,"[{'id': 35, 'name': 'Comedy'}, {'id': 878, 'na...",http://www.kamikazegirls.net/,26275,tt0416220,ja,Shimotsuma Monogatari,"Momoko is an ordinary girl, living an ordinary...",...,2004-05-29,0.0,102.0,"[{'iso_639_1': 'ja', 'name': '日本語'}]",Released,The extraordinary adventures of a Lolita-look ...,Kamikaze Girls,False,7.1,52.0
18832,False,NaN,85000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,62764,tt1667353,en,Mirror Mirror,"After she spends all her money, an evil enchan...",...,2012-03-15,183018522.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The Snow White legend comes alive.,Mirror Mirror,False,5.5,1148.0


##### _Observaciones_
Se observa que en las columnas "cast" y "crew" del dataframe "creditos", se encuentran anidadas en un formato diccionario. Así mismo, las columnas "belongs_to_collection", "genres", "production_companies", "production_countries" y "spoken_languages" del datraframe "movies", también se encuentran anidados en formato diccionario.

In [7]:
# Revisamos la información del dataframe "creditos" para conocer si cuneta con valores nulos.
creditos.info()
# Así mismo se puede observar los tipos de datos que contiene cada columna.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45476 entries, 0 to 45475
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   cast    45476 non-null  object
 1   crew    45476 non-null  object
 2   id      45476 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 1.0+ MB


In [8]:
# Revisamos la información del dataframe "movies" para conocer si cuneta con valores nulos.
movies.info()
# Si bien hay algunos datos nulos, no sería factible eliminarlos ya que podemos tener pocos datos a analizar.
# Así mismo se puede observar los tipos de datos que contiene cada columna.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

In [9]:
# Visualizamos la forma de la estructura del dataframe "creditos".
creditos.shape
# Tiene 45476 filas y 3 columnas.

(45476, 3)

In [10]:
# Visualizamos la forma de la estructura del dataframe "movies".
movies.shape
# Tiene 45466 filas y 24 columnas.

(45466, 24)

## 2. Transformaciones de los dataframes

Algunos campos, como "cast" y "crew" del dataframe "creditos" y  "belongs_to_collection", "genres", "production_companies", "production_countries" y "spoken_languages" del datraframe "movies" están anidados, esto es o bien tienen un diccionario o una lista como valores en cada fila. Se desanidarán para poder utilizarlos y se unirán al dataset de nuevo para hacer alguna de las consultas de la API.

*Crew*

In [11]:
# Observamos el contenido de la columna "crew" del dataframe "creditos", en este contenido, necesitamos el nombre del "Director".
creditos['crew'].head(3).values

array(['[{\'credit_id\': \'52fe4284c3a36847f8024f49\', \'department\': \'Directing\', \'gender\': 2, \'id\': 7879, \'job\': \'Director\', \'name\': \'John Lasseter\', \'profile_path\': \'/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg\'}, {\'credit_id\': \'52fe4284c3a36847f8024f4f\', \'department\': \'Writing\', \'gender\': 2, \'id\': 12891, \'job\': \'Screenplay\', \'name\': \'Joss Whedon\', \'profile_path\': \'/dTiVsuaTVTeGmvkhcyJvKp2A5kr.jpg\'}, {\'credit_id\': \'52fe4284c3a36847f8024f55\', \'department\': \'Writing\', \'gender\': 2, \'id\': 7, \'job\': \'Screenplay\', \'name\': \'Andrew Stanton\', \'profile_path\': \'/pvQWsu0qc8JFQhMVJkTHuexUAa1.jpg\'}, {\'credit_id\': \'52fe4284c3a36847f8024f5b\', \'department\': \'Writing\', \'gender\': 2, \'id\': 12892, \'job\': \'Screenplay\', \'name\': \'Joel Cohen\', \'profile_path\': \'/dAubAiZcvKFbboWlj7oXOkZnTSu.jpg\'}, {\'credit_id\': \'52fe4284c3a36847f8024f61\', \'department\': \'Writing\', \'gender\': 0, \'id\': 12893, \'job\': \'Screenplay\', \'name

In [12]:
# Extraemos el nombre del director de la columna "crew" del dataframe "creditos"
def extract_director(row):
    try:
        crew_list = ast.literal_eval(row['crew'])
        for crew_member in crew_list:
            if crew_member.get('job') == 'Director':
                return crew_member.get('name')
    except (SyntaxError, ValueError):
        pass
    return None

# Aplicamos la función para obtener el nombre del director
creditos['Director'] = creditos.apply(extract_director, axis=1)

creditos.head(5)


,cast,crew,id,Director
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862,John Lasseter
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844,Joe Johnston
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602,Howard Deutch
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357,Forest Whitaker
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862,Charles Shyer


In [13]:
# Comprobamos si los datos extraidos son correctos con los datos originales.
creditos['crew'].head(10).values

array(['[{\'credit_id\': \'52fe4284c3a36847f8024f49\', \'department\': \'Directing\', \'gender\': 2, \'id\': 7879, \'job\': \'Director\', \'name\': \'John Lasseter\', \'profile_path\': \'/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg\'}, {\'credit_id\': \'52fe4284c3a36847f8024f4f\', \'department\': \'Writing\', \'gender\': 2, \'id\': 12891, \'job\': \'Screenplay\', \'name\': \'Joss Whedon\', \'profile_path\': \'/dTiVsuaTVTeGmvkhcyJvKp2A5kr.jpg\'}, {\'credit_id\': \'52fe4284c3a36847f8024f55\', \'department\': \'Writing\', \'gender\': 2, \'id\': 7, \'job\': \'Screenplay\', \'name\': \'Andrew Stanton\', \'profile_path\': \'/pvQWsu0qc8JFQhMVJkTHuexUAa1.jpg\'}, {\'credit_id\': \'52fe4284c3a36847f8024f5b\', \'department\': \'Writing\', \'gender\': 2, \'id\': 12892, \'job\': \'Screenplay\', \'name\': \'Joel Cohen\', \'profile_path\': \'/dAubAiZcvKFbboWlj7oXOkZnTSu.jpg\'}, {\'credit_id\': \'52fe4284c3a36847f8024f61\', \'department\': \'Writing\', \'gender\': 0, \'id\': 12893, \'job\': \'Screenplay\', \'name

In [14]:
creditos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45476 entries, 0 to 45475
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   cast      45476 non-null  object
 1   crew      45476 non-null  object
 2   id        45476 non-null  int64 
 3   Director  44589 non-null  object
dtypes: int64(1), object(3)
memory usage: 1.4+ MB


*Belongs_to_collection*

In [15]:
# Utilizamos ast.literal_eval para evaluar la columna 'belongs_to_collection' como un diccionario de Python
movies['belongs_to_collection'] = movies['belongs_to_collection'].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else 'None')

# Extraemos el nombre de la colección
movies['collection_name'] = movies['belongs_to_collection'].apply(lambda x: x.get('name', '') if isinstance(x, dict) else 'None')

# Imprimimos movies con la columna normalizada
movies.sample(5)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,collection_name
28050,False,None,0,"[{'id': 35, 'name': 'Comedy'}]",http://www.khartonline.com/,27866,tt1420554,en,Kevin Hart: I'm a Grown Little Man,Fresh off the heels of appearing in movies lik...,...,0.0,72.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Kevin Hart: I'm a Grown Little Man,False,7.5,41.0,None
27912,False,None,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10769, 'n...",NaN,69335,tt1039960,pt,Estômago,"In a dog-eat-dog world, Raimundo Nonato has fo...",...,0.0,113.0,"[{'iso_639_1': 'pt', 'name': 'Português'}]",Released,"An adult fable on power, sex and cooking",Estômago: A Gastronomic Story,False,7.9,25.0,None
21824,False,"{'id': 90031, 'name': 'Inner Sanctum mysteries...",0,"[{'id': 14, 'name': 'Fantasy'}, {'id': 9648, '...",NaN,64078,tt0037722,en,The Frozen Ghost,"When a man dies of a heart attack, a stage and...",...,0.0,61.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,The Frozen Ghost,False,6.7,8.0,Inner Sanctum mysteries
11128,False,None,0,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...",NaN,47758,tt0026424,en,The Good Fairy,Luisa Ginglebusher (Sullavan) leaves the orpha...,...,0.0,98.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,The Good Fairy,False,5.0,5.0,None
26569,False,"{'id': 748, 'name': 'X-Men Collection', 'poste...",178000000,"[{'id': 878, 'name': 'Science Fiction'}]",http://www.foxmovies.com/movies/x-men-apocalypse,246655,tt3385516,en,X-Men: Apocalypse,After the re-emergence of the world's first mu...,...,543934787.0,144.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Only the strong will survive,X-Men: Apocalypse,False,6.4,4831.0,X-Men Collection


*Genres*

In [16]:
# Utilizamos ast.literal_eval para evaluar la columna 'genres' como un diccionario de Python
movies['genres'] = movies['genres'].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else 'None')
        
# Extraemos los nombres de los géneros con una función
def extract_genre_names(genres_list):
    genre_names = [genre['name'] for genre in genres_list]
    return ', '.join(genre_names)

# Aplicamos la función a cada elemento de la columna 'genres'
movies['genre'] = movies['genres'].apply(extract_genre_names)

# Imprimimos el DataFrame con la columna de géneros normalizados
movies.sample(5)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,collection_name,genre
10083,False,None,1000000,"[{'id': 18, 'name': 'Drama'}, {'id': 10402, 'n...",NaN,27,tt0411705,en,9 Songs,"Matt, a young glaciologist, soars across the v...",...,66.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,"2 lovers, one summer, and the 9 songs that def...",9 Songs,False,5.1,103.0,None,"Drama, Music, Romance"
38329,False,None,0,[],NaN,93734,tt0371501,en,.hack Liminality: In the Case of Mai Minase,In the first volume of the .hack//Liminality s...,...,45.0,[],Released,NaN,.hack Liminality: In the Case of Mai Minase,False,0.0,1.0,None,
22761,False,None,35000000,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",NaN,144336,tt1742334,en,Sabotage,"In ""Sabotage"", Arnold Schwarzenegger leads an ...",...,110.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Leave no loose ends,Sabotage,False,5.5,560.0,None,"Action, Drama, Thriller, Crime"
33640,False,None,0,"[{'id': 99, 'name': 'Documentary'}]",NaN,259985,tt3597400,en,Just Eat It: A Food Waste Story,"We all love food. As a society, we devour coun...",...,75.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Just Eat It: A Food Waste Story,False,7.8,5.0,None,Documentary
34113,False,None,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",NaN,140648,tt0078976,en,City on Fire,"A pyromaniac, ex-employee of a city oil refine...",...,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,What Happened to Them Could Happen to You ... ...,City on Fire,False,7.0,5.0,None,"Action, Drama"


In [17]:
# Cambiamos los campos vacíos de las columnas "genres"  y "genre" por "None" en el daraframe "movies".
movies['genres'] = movies['genres'].apply(lambda x: 'None' if x == [] else x)
movies['genre'] = movies['genre'].apply(lambda x: 'None' if x == '' else x)

In [18]:
movies['genres'][55]

'None'

In [19]:
# Comprobamos que estos valores estén con la palabra "None".
movies.sample(3)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,collection_name,genre
14359,False,"{'id': 656, 'name': 'Saw Collection', 'poster_...",11000000,"[{'id': 27, 'name': 'Horror'}, {'id': 9648, 'n...",http://www.saw6film.com/,22804,tt1233227,en,Saw VI,"Special Agent Strahm is dead, and Detective Ho...",...,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The Game Has Come Full Circle,Saw VI,False,6.0,755.0,Saw Collection,"Horror, Mystery"
35914,False,None,250000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,54242,tt0009369,en,Mickey,"Mickey, an orphan who has been brought up in a...",...,93.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Mickey,False,7.5,4.0,None,"Comedy, Drama"
25166,False,None,0,"[{'id': 36, 'name': 'History'}, {'id': 10752, ...",http://pwatkins.mnsi.net/forgottenFaces.htm,155130,tt0317522,en,The Forgotten Faces,"“The Forgotten Faces (1961), a film reconstruc...",...,18.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,The Forgotten Faces,False,5.0,1.0,None,"History, War, Drama"


*Production_companies*

In [20]:
# Realizamos una función para normalizar la columna 'production_companies'
def normalize_production_companies(companies_list):
    if isinstance(companies_list, list):  # Si ya es una lista, asumimos que los datos están correctamente formateados
        company_names = [company['name'] for company in companies_list]
        return ', '.join(company_names)
    else:  # Si no es una lista, utilizamos ast.literal_eval para convertir la cadena JSON en una lista
        try:
            companies_list = ast.literal_eval(companies_list)
            if isinstance(companies_list, list):
                company_names = [company['name'] for company in companies_list]
                return ', '.join(company_names)
            else:
                return 'None'
        except (SyntaxError, ValueError):
            return 'None'

# Aplicamos la función a cada elemento de la columna 'production_companies'
movies['company'] = movies['production_companies'].apply(normalize_production_companies)

# Imprimimos movies con la columna de compañías de producción normalizada
movies.sample(5)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,spoken_languages,status,tagline,title,video,vote_average,vote_count,collection_name,genre,company
4706,False,None,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",NaN,42158,tt0080556,en,The Competition,The movie centers on a piano competition whose...,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,If you're really in love... nothing's going to...,The Competition,False,6.0,1.0,None,"Drama, Romance, Music","Columbia Pictures Corporation, Rastar Films"
11280,False,"{'id': 278467, 'name': 'Trilogía de la violenc...",0,"[{'id': 18, 'name': 'Drama'}]",NaN,32761,tt0098327,de,Der siebente Kontinent,Chronicles three years of a middle class famil...,...,"[{'iso_639_1': 'de', 'name': 'Deutsch'}]",Released,NaN,The Seventh Continent,False,7.6,65.0,Trilogía de la violencia de Michael Haneke,Drama,Wega Film
20332,False,None,0,None,NaN,76686,tt1667130,de,The Green Wave,NaN,...,"[{'iso_639_1': 'de', 'name': 'Deutsch'}, {'iso...",NaN,NaN,The Green Wave,False,8.5,2.0,None,None,
34850,False,"{'id': 380170, 'name': 'Flying Guillotine Coll...",0,"[{'id': 28, 'name': 'Action'}, {'id': 10769, '...",NaN,62071,tt0071512,zh,Xue di zi,The Emperor's armies have developed a new weap...,...,"[{'iso_639_1': 'zh', 'name': '普通话'}]",Released,NaN,The Flying Guillotine,False,6.6,5.0,Flying Guillotine Collection,"Action, Foreign",Shaw Brothers
10202,False,None,0,"[{'id': 53, 'name': 'Thriller'}]",NaN,26654,tt0037611,en,Conflict,"Filmed some 18 months before its release, Conf...",...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,SUSPENSE...SUSPICION...MAN-WOMAN DESIRES!,Conflict,False,5.6,14.0,None,Thriller,Warner Bros.


In [21]:
# Cambiamos los campos vacíos de las columnas "genres"  y "genre" por "None" en el daraframe "movies".
movies['production_companies']= movies['production_companies'].apply(lambda x: 'None' if x == '[]' else x)
movies['company'] = movies['company'].apply(lambda x: 'None' if x == '' else x)

*Production_countries*

In [22]:
# Aseguramos que la columna 'production_countries' contenga datos JSON válidos
movies['production_countries'] = movies['production_countries'].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else 'None')

# Realizamos una función para normalizar la columna 'production_countries'
def normalize_production_countries(countries_list):
    if isinstance(countries_list, list):
        country_names = [country['name'] for country in countries_list]
        return ', '.join(country_names)
    else:
        return 'None'

# Aplicamos la función a cada elemento de la columna 'production_countries'
movies['country'] = movies['production_countries'].apply(normalize_production_countries)

# Imprimimos movies con la columna de países de producción normalizada
movies.sample(5)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,status,tagline,title,video,vote_average,vote_count,collection_name,genre,company,country
17682,False,None,0,"[{'id': 27, 'name': 'Horror'}, {'id': 9648, 'n...",NaN,15052,tt0851530,en,The Lodger,Set in Los Angeles with two converging plot li...,...,Released,Everyone is Suspect,The Lodger,False,5.5,42.0,None,"Horror, Mystery, Thriller",Merchant Pacific Corporation,United States of America
6705,False,None,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",NaN,17744,tt0058962,en,The Bedford Incident,Richard Widmark plays a hardened cold-war warr...,...,Released,"""Hunt her down...until she comes up!""",The Bedford Incident,False,7.0,24.0,None,"Action, Drama, Thriller, War","Columbia Pictures Corporation, Bedford Product...","United Kingdom, United States of America"
782,False,None,0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,49963,tt0118113,en,Walking and Talking,Things have been tough lately for Amelia. Her ...,...,Released,A movie for everyone who wants to get married ...,Walking and Talking,False,6.6,8.0,None,"Comedy, Drama, Romance","Channel Four Films, Pandora Filmproduktion, Ze...","Germany, United Kingdom, United States of America"
29879,False,None,7500000,"[{'id': 28, 'name': 'Action'}]",NaN,326284,tt3488328,en,Pound of Flesh,In China to donate his kidney to his dying nie...,...,Released,The black market is about to get bloody,Pound of Flesh,False,5.4,64.0,None,Action,"Odyssey Media, Rodin Entertainment, Ace Film C...",Canada
12293,False,None,0,"[{'id': 18, 'name': 'Drama'}, {'id': 35, 'name...",NaN,1488,tt0461804,zh,綠草地,Bilike has never seen a ping-pong ball before....,...,Released,NaN,Mongolian Ping Pong,False,0.0,0.0,None,"Drama, Comedy",TV Productions LTD And Beijing Hop Culture Co,China


In [23]:
movies['production_countries']= movies['production_countries'].apply(lambda x: 'None' if x == [] else x)
movies['country'] = movies['country'].apply(lambda x: 'None' if x == '' else x)

*Spoken_languages*

In [24]:
# Aseguramos que la columna 'spoken_languages' contenga datos JSON válidos
movies['spoken_languages'] = movies['spoken_languages'].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else 'None')

# Realizamos una función para normalizar la columna "spoken_languages"
def normalize_spoken_languages(languages_list):
    if isinstance(languages_list, list):
        language_names = [language['name'] for language in languages_list]
        return ', '.join(language_names)
    else:
        return 'None'

# Aplicamos la función a cada elemento de la columna 'spoken_languages'
movies['spoken_language'] = movies['spoken_languages'].apply(normalize_spoken_languages)

# Imprimimos movies con la columna de lenguajes normalizada
movies.sample(5)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,tagline,title,video,vote_average,vote_count,collection_name,genre,company,country,spoken_language
29459,False,None,1000000,"[{'id': 18, 'name': 'Drama'}, {'id': 12, 'name...",http://www.frozensouth.com/,201132,tt2361700,en,Antarctica: A Year on Ice,Filling the giant screen with stunning time-la...,...,Step Outside The Box,Antarctica: A Year on Ice,False,7.2,31.0,None,"Drama, Adventure, Documentary",Antzworks,New Zealand,English
6716,False,None,0,"[{'id': 18, 'name': 'Drama'}, {'id': 53, 'name...",NaN,40945,tt0314039,en,The Event,An intense relationship drama that takes the f...,...,NaN,The Event,False,6.7,3.0,None,"Drama, Thriller",None,None,English
37384,False,None,0,"[{'id': 18, 'name': 'Drama'}]",NaN,157033,tt2493318,en,Die Welt,Die Welt is an audacious hybrid between fictio...,...,NaN,Die Welt,False,0.0,0.0,None,Drama,None,None,
2723,False,None,0,"[{'id': 35, 'name': 'Comedy'}]",NaN,4338,tt0125211,en,I Woke Up Early The Day I Died,A mental patient escapes from the looney bin i...,...,NaN,I Woke Up Early The Day I Died,False,4.7,3.0,None,Comedy,None,United States of America,English
44003,False,None,5000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 28, 'nam...",NaN,41978,tt1438463,ru,Каникулы строгого режима,Two escaped convicts pose as children's summer...,...,NaN,High Security Vacation,False,6.4,12.0,None,"Comedy, Action","Film Direction, 20th Century Fox Russia",Russia,Pусский


In [25]:
movies['spoken_languages']= movies['spoken_languages'].apply(lambda x: 'None' if x == [] else x)
movies['spoken_language']= movies['spoken_language'].apply(lambda x: 'None' if x == '' else x)

## 3. Eliminamos las columnas que no están desanidadas y las que no nos servirán

In [26]:
creditos.drop(columns=['cast','crew'],inplace=True)
creditos.head()

,id,Director
0,862,John Lasseter
1,8844,Joe Johnston
2,15602,Howard Deutch
3,31357,Forest Whitaker
4,11862,Charles Shyer


In [27]:
movies.drop(columns=['belongs_to_collection', 'genres', 'production_companies', 'production_countries', 'spoken_languages', 'video', 'imdb_id', 'adult', 'original_title', 'poster_path', 'homepage',], inplace = True)
movies.head()

,budget,id,original_language,overview,popularity,release_date,revenue,runtime,status,tagline,title,vote_average,vote_count,collection_name,genre,company,country,spoken_language
0,30000000,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,81.0,Released,NaN,Toy Story,7.7,5415.0,Toy Story Collection,"Animation, Comedy, Family",Pixar Animation Studios,United States of America,English
1,65000000,8844,en,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249.0,104.0,Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,None,"Adventure, Fantasy, Family","TriStar Pictures, Teitler Film, Interscope Com...",United States of America,"English, Français"
2,0,15602,en,A family wedding reignites the ancient feud be...,11.7129,1995-12-22,0.0,101.0,Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0,Grumpy Old Men Collection,"Romance, Comedy","Warner Bros., Lancaster Gate",United States of America,English
3,16000000,31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,1995-12-22,81452156.0,127.0,Released,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0,None,"Comedy, Drama, Romance",Twentieth Century Fox Film Corporation,United States of America,English
4,0,11862,en,Just when George Banks has recovered from his ...,8.387519,1995-02-10,76578911.0,106.0,Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0,Father of the Bride Collection,Comedy,"Sandollar Productions, Touchstone Pictures",United States of America,English


## 4. Duplicamos valores para mejorar la consulta de desanidados

##### _Observaciones_
En las columnas desanidas del dataframe "movies", hay algunos valores separados por comas, por ejemplo "genres" que contiene (Animation, Comedy, Family).
Es por eso que haremos un análsis con valores únicos y posteriormente duplicaremos las filas para que de desagreguen esos valores de cada columna.

Imprimimos los datos únicos de los datos desanidados y la cantidad de estos

In [28]:
# Al imprimir los valores únicos, nos damos cuenta que hay valores únicos que contienen a partir de 2 a más características, como lo explicado en genres anteriormente.
print (movies.collection_name.unique())
print(movies.genre.unique())
print(movies.company.unique())
print(movies.country.unique())
print(movies.spoken_language.unique())


['Toy Story Collection' 'None' 'Grumpy Old Men Collection' ...
 'Ducobu Collection' 'Mister Blot Collection' 'Red Lotus Collection']
['Animation, Comedy, Family' 'Adventure, Fantasy, Family'
 'Romance, Comedy' ... 'Crime, Comedy, Action, Family'
 'Action, Mystery, Thriller, Horror' 'Family, Animation, Romance, Comedy']
['Pixar Animation Studios'
 'TriStar Pictures, Teitler Film, Interscope Communications'
 'Warner Bros., Lancaster Gate' ...
 'Westdeutscher Rundfunk (WDR), Working Title Films, 20th Century Fox Television, CanWest Global Communications'
 'Sine Olivia' 'Yermoliev']
['United States of America' 'Germany, United States of America'
 'United Kingdom, United States of America' ...
 'Poland, Czech Republic, Slovakia' 'Cuba, Germany, Spain'
 'Egypt, Italy, United States of America']
['English' 'English, Français' 'English, Español' ... 'svenska, Deutsch'
 'العربية, Polski' 'Fulfulde, English']


In [29]:
# Imprimiremos la cantidad de valores únicos para ver el cambio posteriormente al hacer el duplicado.
print ('Cantidad de collection_name:',movies.collection_name.nunique())
print('Cantidad de genre:',movies.genre.nunique())
print('Cantidad de company:',movies.company.nunique())
print('Cantidad de country:',movies.country.nunique())
print('Cantidad de spoken_language:',movies.spoken_language.nunique())

Cantidad de collection_name: 1696
Cantidad de genre: 4069
Cantidad de company: 22671
Cantidad de country: 2390
Cantidad de spoken_language: 1842


Crear registros duplicados

In [30]:
# Se crearán filas duplicadas ya que al desanidar hay dos datos en una fila, Ejemplo: 'genre', donde algunos tienen 'Animation, Comedy', estos valores se desagregarán.

# Se crean nuevas columnas 
movies['collection_names'] = movies['collection_name'].str.split(', ')
movies['genres'] = movies['genre'].str.split(', ')
movies['companies'] = movies['company'].str.split(', ')
movies['countries'] = movies['country'].str.split(', ')
movies['languages'] = movies['spoken_language'].str.split(', ')

# Se duplican los registros
movies=movies.explode('collection_names').reset_index(drop=True)
movies=movies.explode('genres').reset_index(drop=True)
movies=movies.explode('companies').reset_index(drop=True)
movies=movies.explode('countries').reset_index(drop=True)
movies=movies.explode('languages').reset_index(drop=True)

#Se eliminan las columnas originales
movies.drop('collection_name',axis=1,inplace=True)
movies.drop('genre',axis=1,inplace=True)
movies.drop('company',axis=1,inplace=True)
movies.drop('country',axis=1,inplace=True)
movies.drop('spoken_language',axis=1,inplace=True)

In [31]:
# Comparamos con el anterior código para ver si hubieron cambios
print (movies.collection_names.unique())
print(movies.genres.unique())
print(movies.companies.unique())
print(movies.countries.unique())
print(movies.languages.unique())

['Toy Story Collection' 'None' 'Grumpy Old Men Collection' ...
 'Ducobu Collection' 'Mister Blot Collection' 'Red Lotus Collection']
['Animation' 'Comedy' 'Family' 'Adventure' 'Fantasy' 'Romance' 'Drama'
 'Action' 'Crime' 'Thriller' 'Horror' 'History' 'Science Fiction'
 'Mystery' 'War' 'Foreign' 'None' 'Music' 'Documentary' 'Western'
 'TV Movie' 'Carousel Productions' 'Vision View Entertainment'
 'Telescene Film Group Productions' 'Aniplex' 'GoHands' 'BROSTA TV'
 'Mardock Scramble Production Committee' 'Sentai Filmworks'
 'Odyssey Media' 'Pulser Productions' 'Rogue State' 'The Cartel']
['Pixar Animation Studios' 'TriStar Pictures' 'Teitler Film' ...
 'Pirie Productions' 'Sine Olivia' 'Yermoliev']
['United States of America' 'Germany' 'United Kingdom' 'France' 'Italy'
 'Spain' 'China' 'Australia' 'South Africa' 'Canada' 'None' 'Switzerland'
 'Belgium' 'Japan' 'Iran' 'Netherlands' 'Hong Kong' 'Tunisia' 'Ireland'
 'Dominican Republic' 'Croatia' 'Russia' 'Macedonia' 'Austria' 'Taiwan'
 'Ne

In [32]:
print ('Cantidad de collection_names:',movies.collection_names.nunique())
print('Cantidad de genres:',movies.genres.nunique())
print('Cantidad de companies:',movies.companies.nunique())
print('Cantidad de countries:',movies.countries.nunique())
print('Cantidad de languages:',movies.languages.nunique())

Cantidad de collection_names: 1721
Cantidad de genres: 33
Cantidad de companies: 23567
Cantidad de countries: 161
Cantidad de languages: 76


In [33]:
# Observamos que las filas se duplicaron y se desagregaron los datos separados por comas.
movies.head(10)

,budget,id,original_language,overview,popularity,release_date,revenue,runtime,status,tagline,title,vote_average,vote_count,collection_names,genres,companies,countries,languages
0,30000000,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,81.0,Released,NaN,Toy Story,7.7,5415.0,Toy Story Collection,Animation,Pixar Animation Studios,United States of America,English
1,30000000,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,81.0,Released,NaN,Toy Story,7.7,5415.0,Toy Story Collection,Comedy,Pixar Animation Studios,United States of America,English
2,30000000,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,81.0,Released,NaN,Toy Story,7.7,5415.0,Toy Story Collection,Family,Pixar Animation Studios,United States of America,English
3,65000000,8844,en,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249.0,104.0,Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,None,Adventure,TriStar Pictures,United States of America,English
4,65000000,8844,en,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249.0,104.0,Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,None,Adventure,TriStar Pictures,United States of America,Français
5,65000000,8844,en,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249.0,104.0,Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,None,Adventure,Teitler Film,United States of America,English
6,65000000,8844,en,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249.0,104.0,Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,None,Adventure,Teitler Film,United States of America,Français
7,65000000,8844,en,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249.0,104.0,Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,None,Adventure,Interscope Communications,United States of America,English
8,65000000,8844,en,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249.0,104.0,Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,None,Adventure,Interscope Communications,United States of America,Français
9,65000000,8844,en,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249.0,104.0,Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,None,Fantasy,TriStar Pictures,United States of America,English


## 5. Valores Nulos

Los valores nulos de los campos revenue, budget deben ser rellenados por el número 0.

In [34]:
# Rellenamos los valores faltantes en 'revenue' y 'budget' con el valor 0.
movies['revenue'].fillna(0, inplace=True)
movies['budget'].fillna(0, inplace=True)

# Contamos la cantidad de valores nulos en 'revenue' y 'budget'
print('Valores nulos en revenue',movies['revenue'].isna().sum())
print('Valores nulos en budget',movies['revenue'].isna().sum())

Valores nulos en revenue 0
Valores nulos en budget 0


Los valores nulos del campo release date deben eliminarse.

In [35]:
# Eliminamos las filas con valores nulos de 'realease_date'
movies=movies.dropna(subset=['release_date'])

# Contamos la cantidad de valores nulos en 'realease_date'
print('Valores nulos en realease_date:', movies['release_date'].isna().sum())

Valores nulos en realease_date: 0


Los valores en la columna "title" del dataframe "movies" cuentan con vacíos

In [36]:
# Observamos la cantidad de valores en la columna "title"
movies['title'].info()

<class 'pandas.core.series.Series'>
Index: 422016 entries, 0 to 422194
Series name: title
Non-Null Count   Dtype 
--------------   ----- 
422004 non-null  object
dtypes: object(1)
memory usage: 6.4+ MB


In [37]:
# Aplicamos una máscara para verificar valores vacíos
mask = movies['title'] > ''
movies = movies.loc[mask]

In [38]:
# Observamos que se eliminaron algunos valores vacíos
movies['title'].info()

<class 'pandas.core.series.Series'>
Index: 422004 entries, 0 to 422194
Series name: title
Non-Null Count   Dtype 
--------------   ----- 
422004 non-null  object
dtypes: object(1)
memory usage: 6.4+ MB


## 6. Modificación de fecha

De haber fechas, deberán tener el formato AAAA-mm-dd, además deberán crear la columna release_year donde extraerán el año de la fecha de estreno.

In [39]:
# Transformamos 'release_date' en el formato 'aaaa-mm-dd', en caso el formato sea incorrecto.
movies['release_date'] = pd.to_datetime(movies['release_date'], errors='coerce').dt.strftime('%Y-%m-%d')

# Creamos la columna 'release_year' extrayendo el año de 'release_date'.
movies['release_year'] = pd.to_datetime(movies['release_date'], errors='coerce').dt.year
movies['release_year'] = movies['release_year'].astype('Int64')

In [40]:
movies[['release_date','release_year']]

,release_date,release_year
0,1995-10-30,1995
1,1995-10-30,1995
2,1995-10-30,1995
3,1995-12-15,1995
4,1995-12-15,1995
...,...,...
422190,2003-08-01,2003
422191,2003-08-01,2003
422192,2003-08-01,2003
422193,1917-10-21,1917


In [41]:
# Observamos que la columna "release_date" del dataframe "movies" es una fecha pero está en formato objeto.
movies['release_date'].info()

<class 'pandas.core.series.Series'>
Index: 422004 entries, 0 to 422194
Series name: release_date
Non-Null Count   Dtype 
--------------   ----- 
422004 non-null  object
dtypes: object(1)
memory usage: 6.4+ MB


In [42]:
# Convertimos la columna "release_date" del dataframe "movies" en formato fecha.
movies['release_date'] = pd.to_datetime(movies['release_date'])

In [43]:
# Observamos que la columna "release_date" del dataframe "movies" está en formato fecha.
movies['release_date'].info()

<class 'pandas.core.series.Series'>
Index: 422004 entries, 0 to 422194
Series name: release_date
Non-Null Count   Dtype         
--------------   -----         
422004 non-null  datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 6.4 MB


In [44]:
# Borramos nulos de 'release_date' y 'release_year'
movies=movies.dropna(subset=['release_date'])
movies=movies.dropna(subset=['release_year'])

# Contamos la cantidad de valores nulos en 'realease_date' y 'release_year'
print('Valores nulos en realease_date:', movies['release_date'].isna().sum())
print('Valores nulos en realease_year:', movies['release_year'].isna().sum())

Valores nulos en realease_date: 0
Valores nulos en realease_year: 0


In [45]:
# Observamos la cantidad de nulos del dataframe "movies"
movies.isna().sum()

budget                    0
id                        0
original_language        96
overview               2495
popularity                0
release_date              0
revenue                   0
runtime                 544
status                  143
tagline              193173
title                     0
vote_average              0
vote_count                0
collection_names          0
genres                    0
companies                 0
countries                 0
languages                 0
release_year              0
dtype: int64

In [46]:
# Observamos la cantidad de nulos del dataframe "creditos"
creditos.isna().sum()

id            0
Director    887
dtype: int64

## 7. Creación de columna retorno de inversión

Crear la columna con el retorno de inversión, llamada return con los campos revenue y budget, dividiendo estas dos últimas revenue / budget, cuando no hay datos disponibles para calcularlo, deberá tomar el valor 0.

In [48]:
# Convertimos 'revenue' y 'budget' a tipo float
movies['revenue'] = pd.to_numeric(movies['revenue'])
movies['budget'] = pd.to_numeric(movies['budget'])

# Calculamos el retorno de inversión
movies['return'] = movies['revenue'].divide(movies['budget']).replace([np.inf, -np.inf],0).fillna(0)
movies[['return']]


,return
0,12.451801
1,12.451801
2,12.451801
3,4.043035
4,4.043035
...,...
422190,0.000000
422191,0.000000
422192,0.000000
422193,0.000000


## 8. Unimos el dataframe "creditos" con "movies"

In [49]:
# Tienen en común id.
movies.head(2)

,budget,id,original_language,overview,popularity,release_date,revenue,runtime,status,tagline,title,vote_average,vote_count,collection_names,genres,companies,countries,languages,release_year,return
0,30000000,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,81.0,Released,NaN,Toy Story,7.7,5415.0,Toy Story Collection,Animation,Pixar Animation Studios,United States of America,English,1995,12.451801
1,30000000,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,81.0,Released,NaN,Toy Story,7.7,5415.0,Toy Story Collection,Comedy,Pixar Animation Studios,United States of America,English,1995,12.451801


In [50]:
# Tienen en común id.
creditos.head(1)

,id,Director
0,862,John Lasseter


In [51]:
# Convertimos la columna 'id' de "creditos" y "movies" a tipo int64 para que coincidan entre ambos.
creditos['id'] = creditos['id'].astype('int64')
movies['id'] = movies['id'].astype('int64')

# Unimos los dataframes en función de la columna 'id', creando un nuevo dataframe "datos".
datos = pd.merge(movies, creditos, on='id')


In [52]:
# Observamos que los datos se hayan unido.
datos.sample(3)

,budget,id,original_language,overview,popularity,release_date,revenue,runtime,status,tagline,...,vote_average,vote_count,collection_names,genres,companies,countries,languages,release_year,return,Director
362665,0,329829,en,"A portrait of the brilliant, extravagant Krist...",9.227292,2015-12-11,0.0,106.0,Released,NaN,...,6.4,37.0,None,Romance,Triptych Media,Finland,Deutsch,2015,0.0,Mika Kaurismäki
181689,0,39286,en,"A comic drama set on a Mediterranean island, w...",0.550246,2007-01-11,0.0,110.0,Released,NaN,...,6.2,3.0,None,Romance,Ingenious Film Partners,Spain,Español,2007,0.0,Francesca Joseph
248037,0,13648,ja,A zombie attack brings chaos to Harvardville A...,8.156555,2008-02-13,0.0,97.0,Released,NaN,...,6.3,248.0,Resident Evil: Biohazard,Horror,Capcom,Japan,English,2008,0.0,Makoto Kamiya


In [53]:
# Observamps los datos vacíos del nuevo datarframe "datos".
datos.isna().sum()

budget                    0
id                        0
original_language        96
overview               2495
popularity                0
release_date              0
revenue                   0
runtime                 544
status                  143
tagline              195330
title                     0
vote_average              0
vote_count                0
collection_names          0
genres                    0
companies                 0
countries                 0
languages                 0
release_year              0
return                    0
Director               1512
dtype: int64

In [54]:
# Cambiamos los datos nulos del dataframe por 'None'.
datos = datos.where(pd.notna(datos), 'None')

In [55]:
# Verificamos que los datos estén parejos.
datos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 424431 entries, 0 to 424430
Data columns (total 21 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   budget             424431 non-null  int64         
 1   id                 424431 non-null  int64         
 2   original_language  424431 non-null  object        
 3   overview           424431 non-null  object        
 4   popularity         424431 non-null  object        
 5   release_date       424431 non-null  datetime64[ns]
 6   revenue            424431 non-null  float64       
 7   runtime            424431 non-null  object        
 8   status             424431 non-null  object        
 9   tagline            424431 non-null  object        
 10  title              424431 non-null  object        
 11  vote_average       424431 non-null  float64       
 12  vote_count         424431 non-null  float64       
 13  collection_names   424431 non-null  object  

## 9. Exportamos el dataframe "datos" a un archivo .csv

In [56]:
# Obtenemos el archivo con el ETL aplicado
datos.to_csv("datos_finales.csv", index=False)

## 10. Realizamos un dataset con los valores necesarios para las consultas en la API y el Modelo de recomendación

In [57]:
# Realizaremos un dataset con los valores necesarios ya que el peso del último dataset exportado es muy grande.
# Leemos el dataset "datos".
datos = pd.read_csv('datos_finales.csv', sep=',')

In [58]:
datos.head(2)

,budget,id,original_language,overview,popularity,release_date,revenue,runtime,status,tagline,...,vote_average,vote_count,collection_names,genres,companies,countries,languages,release_year,return,Director
0,30000000,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,81.0,Released,NaN,...,7.7,5415.0,Toy Story Collection,Animation,Pixar Animation Studios,United States of America,English,1995,12.451801,John Lasseter
1,30000000,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,81.0,Released,NaN,...,7.7,5415.0,Toy Story Collection,Comedy,Pixar Animation Studios,United States of America,English,1995,12.451801,John Lasseter


In [62]:
columnas=['id','budget','original_language','popularity','release_date','revenue','runtime','title', 'vote_average', 'vote_count', 'collection_names', 'genres', 'companies', 'countries', 'languages', 'release_year', 'return', 'Director' ]

In [63]:
data_movies= datos[columnas]

In [64]:
data_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 424431 entries, 0 to 424430
Data columns (total 18 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   id                 424431 non-null  int64  
 1   budget             424431 non-null  int64  
 2   original_language  424335 non-null  object 
 3   popularity         424431 non-null  float64
 4   release_date       424431 non-null  object 
 5   revenue            424431 non-null  float64
 6   runtime            423887 non-null  float64
 7   title              424431 non-null  object 
 8   vote_average       424431 non-null  float64
 9   vote_count         424431 non-null  float64
 10  collection_names   55238 non-null   object 
 11  genres             421287 non-null  object 
 12  companies          397322 non-null  object 
 13  countries          413069 non-null  object 
 14  languages          411960 non-null  object 
 15  release_year       424431 non-null  int64  
 16  re

In [65]:
data_movies.to_csv("datos_main.csv", index=False)